In [1]:
import urllib3
from elasticsearch import Elasticsearch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Configuring ElasticSearch

In [2]:
# Elastic search configuation

es = Elasticsearch(hosts=["https://localhost:9200"],basic_auth=('elastic','bvxL*B3U4Ks8yk2bETD6'),verify_certs=False, ssl_show_warn=False)
es.ping()

True

### Creating index

In [3]:
#creating index
es.indices.create(index="first_index")

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'first_index'})

In [4]:
#looking if the index exists

es.indices.exists(index="first_index")

HeadApiResponse(True)

### Deleting an index

In [5]:
# delete index

es.indices.delete(index="first_index")

ObjectApiResponse({'acknowledged': True})

In [6]:
es.indices.exists(index="first_index")

HeadApiResponse(False)

### Inserting and getting data 

In [7]:
# Inserting data 

doc_1 = {"city": "Paris", "country": "France"}
doc_2 = {"city": "Vienna", "country": "Austria"}
doc_3 = {"city": "London", "country": "England"}

es.index(index="cities",id=1, document=doc_1)


ObjectApiResponse({'_index': 'cities', '_id': '1', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 3})

In [8]:
es.index(index="cities",id=2, document=doc_2)

ObjectApiResponse({'_index': 'cities', '_id': '2', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 3})

In [9]:
es.index(index="cities",id=3, document=doc_3)

ObjectApiResponse({'_index': 'cities', '_id': '3', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 3})

In [11]:
# Getting the data

res = es.get(index="cities",id=3)
res

ObjectApiResponse({'_index': 'cities', '_id': '3', '_version': 2, '_seq_no': 5, '_primary_term': 3, 'found': True, '_source': {'city': 'London', 'country': 'England'}})

In [12]:
# Get the data which is prestent in the _source key 

res["_source"]

{'city': 'London', 'country': 'England'}

### Search query and matching documents 

In [13]:
#Creating out data 

doc_1 = {"sentence": "Hack Covid-19 is amazing1"}
doc_1 = {"sentence": "Hack Quarantine is stunning!"}

es.index(index="english", id=1, document= doc_1)
es.index(index="english", id=2, document= doc_2)

ObjectApiResponse({'_index': 'english', '_id': '2', '_version': 2, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 3})

In [14]:
#creatingh our query 
## Match query 

body = {
        "match":{
            "sentence":"Hack"
        }
    }

res = es.search(index="english", query=body)
res

ObjectApiResponse({'took': 970, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.2876821, 'hits': [{'_index': 'english', '_id': '1', '_score': 0.2876821, '_source': {'sentence': 'Hack Quarantine is stunning!'}}]}})

In [15]:
body = {
        "match":{
            "sentence":"Quarantine"
        }
    }

res = es.search(index="english", query=body)
res

ObjectApiResponse({'took': 6, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.2876821, 'hits': [{'_index': 'english', '_id': '1', '_score': 0.2876821, '_source': {'sentence': 'Hack Quarantine is stunning!'}}]}})

In [41]:
body= {
    "from":0,
    "size":2,
    "query": {
        "match":{
            "sentence": "Hack Quarantine"
        }
    }
}

res= es.search(index="english", body=body)
res

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.5753642,
  'hits': [{'_index': 'english',
    '_type': 'senetence',
    '_id': '1',
    '_score': 0.5753642,
    '_source': {'sentence': 'Hack Quarantine is stunning!'}}]}}

### Combining Queries